In [58]:
import syllables 
import pandas as pd
import random
import numpy as np
import re
import language_tool_python
from IPython.display import display
import nltk
from nltk import pos_tag
import timeit

In [59]:
import language_tool_python

In [60]:
tool = language_tool_python.LanguageTool('en-US')

In [61]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Vaishnavi D\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [62]:
# !pip install gingerit

In [63]:
with open("Dataset_poems.txt", "r",encoding="utf8") as file:
    # Trial_data.txt
    # Dataset_poems.txt
    temp = []
    for line in file:
        line1 = re.sub('[^a-zA-Z ]', '', line)
        line1 = line1.split()
        if line1 != ['', ''] and line1 != ['']:
            for i in line1:
                if i == '':
                    continue
                else:
                    temp.append(i)
words_df = temp[:5000] 

In [64]:
tokens_tag = pos_tag(words_df)
dic_tag = {}
for tup in tokens_tag:
  if tup[1] not in dic_tag.keys():
    dic_tag[tup[1]] = [tup[0]]
  else:
    temp = dic_tag[tup[1]] + [tup[0]]
    dic_tag[tup[1]] = temp
# print(dic_tag.keys())
z = list(dic_tag.values())
words_df2 = []
for i in range(len(z)):
  words_df2.extend(z[i])
# print(words_df2)
words_df = words_df2

In [119]:
class poem_generation:
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)

    def select_random_words(self,words_df):
        words_array1 = []
        while len(words_array1) <= 15:
            random_number = random.randint(0, len(words_df)-1)
            if words_df[random_number] != 0:
                words_array1.append(words_df[random_number])
        # print(len(words_array1))

        return words_array1

    def create_poems(self,arr):
        poem = [0 for _ in range(4)]
        for i in range(1,5):
            temp =  arr[4*(i-1):4*i]
            poem[i-1] = temp
            # print("HERE:",temp)
            # print(type(temp))
            poem[i-1]=' '.join(poem[i-1])
            # print(poem[i-1])
        # print("=========================================================")
        return poem

    def store_syllables(self,arr):
        a = [0 for _ in range(4)]
        temp = 0
        j = 0
        for words,i in zip(arr,range(0,16)):
            temp = temp + syllables.estimate(words)
            if i in [3,7,11,15]:
                a[j] = temp
                # print(a[j])
                temp = 0
                j += 1
        return a

    def Grammar_checker(self,text):
        error = 0
        poem_error = []
        # tool = language_tool_python.LanguageTool('en-US')
        for lines in text:
            matches = tool.check(lines)
            if len(matches)>0:
                error = error + matches[0].errorLength
        poem_error.append(error)
        return poem_error

    def crossover(self,p1,p2,cross):
        part1 = []
        part2 = []
        part3 = []
        child1 = []
        child2 = []
        j = 0
        k = 0
        random_number1 = sorted([random.randint(1, 15) for i in range(cross)])
        random_number2 = sorted([random.randint(1, 15) for i in range(cross)])
        for i in random_number1:
            part1.append(p1[j:i])
            j = i
        for i in random_number2:
            part2.append(p2[k:i])
            k = i
        part1.append(p1[j:])
        part2.append(p2[k:])
        part2.extend(part1)
        random.shuffle(part2)
        for i in range(len(part2)):
            part3.extend(part2[i])
        for i in range(int(len(part3)/2)):
            child1.append(part3[i])
        for i in range(int(len(part3)/2),int(len(part3))):
            child2.append(part3[i])
        # if child1 == p1 or child1 == p2 or child2 == p1 or child2 == p2:
        #     child1, child2,child1_poem,child2_poem = self.crossover(child1,child2,cross)
        child1_poem = self.create_poems(child1)
        child2_poem = self.create_poems(child2)
        return child1,child2,child1_poem,child2_poem

    def tournament(self,Population,m,best):
        from_best = len(best)
        # print("best:",best)
        except_best = m - from_best
        # df = pd.DataFrame()
        #selected_polulation = pd.DataFrame(columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError","word_array"])
        ######################### when no syllables rule
        # selected_polulation = Population.iloc[:nn]
        # random_number = random.sample(list(Population.iloc[nn:].index),p)
        # Population.sort_values(by=['GrammarError'], inplace=True)
        # Population2 = Population[~Population.isin(selected_polulation).all(1)]
        ######################### when syllables rule
        if len(best) >0:
            selected_polulation = []
            words_array_selected = []
            for b in best:
                c = Population.iloc[b].values[:-1]
                d = Population.iloc[b].values[-1]
                selected_polulation.append(c)
                words_array_selected.append(d)
            words_array_selected = pd.DataFrame(dict(zip(["words_array"],[words_array_selected])))
            selected_polulation = pd.DataFrame(selected_polulation,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError"])
            selected_polulation = pd.concat([selected_polulation,words_array_selected],axis=1)
            # print("poems selected by user")
            # display(selected_polulation)
            Population2 = Population[~Population.isin(selected_polulation).all(1)]
            Population2 = Population2.sort_values(by=['GrammarError'])
            selected_polulation2 = Population2[(Population2["Syllables1"] == "5")&(Population2["Syllables2"] == "6")&(Population2["Syllables3"] == "6")&(Population2["Syllables4"] == "5")]
            # selected_polulation2 = Population2[(Population2["Syllables1"] >= "3")]
            # print("grammar sorted poems by syll")
            # display(selected_polulation2)
            from_sy = len(selected_polulation2.index)
            if from_sy >= int(except_best/2):
                from_sy = int(except_best/2)
            else:
                from_sy = from_sy
            from_random = except_best - from_sy
            selected_polulation2 = selected_polulation2.iloc[:from_sy]
            selected_polulation = pd.concat([selected_polulation,selected_polulation2],axis=0)
            selected_polulation = selected_polulation.reset_index(drop=True)
            display(selected_polulation)
            display(Population2)
            Population2 = Population2[~Population2.isin(selected_polulation).all(1)]
            Population2 = Population2.reset_index(drop=True)
            selected_polulation = pd.concat([selected_polulation,Population2.iloc[:from_random]],axis=0)
            selected_polulation = selected_polulation.reset_index(drop=True)
            # print("SELECTED")
            # display(selected_polulation)
            words_array_selected = list(selected_polulation["words_array"])
            # print("TYPE",type(selected_polulation["words_array"]),words_array_selected)
            return selected_polulation,words_array_selected
        else:
            Population2 = Population.sort_values(by=['GrammarError'])
            random = int(except_best/3)
            grammar = except_best - random
            selected_polulation = Population2[:grammar]
            Population2 = Population2[~Population2.isin(selected_polulation).all(1)]
            selected_polulation2 = Population2[:random]
            selected_polulation = pd.concat([selected_polulation,selected_polulation2],axis=0)
            selected_polulation = selected_polulation.reset_index(drop=True)
            # print("SELECTED")
            # display(selected_polulation)
            words_array_selected = list(selected_polulation["words_array"])
            # print("TYPE",type(selected_polulation["words_array"]),words_array_selected)
            return selected_polulation,words_array_selected



    def create_set_of_poems(self,n):
            poem_population = []
            syllables_count = []
            grammar_error = []
            words_array_population = []
            for i in range(n):
                words_array = self.select_random_words(words_df)
                poem = self.create_poems(words_array)
                sum_syllables = self.store_syllables(words_array)
                poem_error = self.Grammar_checker(poem)
                poem_population.append(poem)
                syllables_count.append(sum_syllables)
                grammar_error.append(poem_error)
                words_array_population.append(words_array)
            words_array2 = pd.DataFrame(dict(zip(["words_array"],[words_array_population])))
            Final = np.hstack((poem_population,syllables_count,grammar_error))
            Population_data = pd.DataFrame(Final,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError"])
            Pop_data = pd.concat([Population_data,words_array2],axis=1)
            # print()
            # print("PARENTS")
            # display(Pop_data)

            return Pop_data

    def child_poems(self,words_array,cross):
        child_population = []
        syllables_count = []
        grammer_error = []
        words_array_child = []
        # print("words_array",words_array)
        # print(type(words_array))
        if len(words_array)%2 == 2:
            for i in range(0,len(words_array)-1):
                child1,child2,child1_poem,child2_poem = self.crossover(words_array[i],words_array[i+1],cross)
                child_population.append(child1_poem)
                child_population.append(child2_poem)
                sum_syllables1 = self.store_syllables(child1)
                sum_syllables2 = self.store_syllables(child2)
                poem_error1 = self.Grammar_checker(child1_poem)
                poem_error2 = self.Grammar_checker(child2_poem)
                syllables_count.append(sum_syllables1)
                syllables_count.append(sum_syllables2)
                grammer_error.append(poem_error1)
                grammer_error.append(poem_error2)
                words_array_child.append(child1)
                words_array_child.append(child2)
        else:
            for i in range(0,len(words_array)-1,2):
                # print(words_array[i],words_array[i+1])
                child1,child2,child1_poem,child2_poem = self.crossover(words_array[i],words_array[i+1],cross)
                child_population.append(child1_poem)
                child_population.append(child2_poem)
                sum_syllables1 = self.store_syllables(child1)
                sum_syllables2 = self.store_syllables(child2)
                poem_error1 = self.Grammar_checker(child1_poem)
                poem_error2 = self.Grammar_checker(child2_poem)
                syllables_count.append(sum_syllables1)
                syllables_count.append(sum_syllables2)
                grammer_error.append(poem_error1)
                grammer_error.append(poem_error2)
                words_array_child.append(child1)
                words_array_child.append(child2)
        words_array_child = pd.DataFrame(dict(zip(["words_array"],[words_array_child])))
        Final1 = np.hstack((child_population,syllables_count,grammer_error))
        child_data = pd.DataFrame(Final1,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError"])
        child_data = pd.concat([child_data,words_array_child],axis=1)
        child_data = child_data.reset_index(drop=True)
        print()
        # print("CHILD")
        # display(child_data)
        return child_data

    def selection_from_child_parents(self,x,child_data,selected_Pop_data):
        reduce = (len(list(child_data.index)) + len(list(selected_Pop_data.index))) - x
        n = int(reduce/3)
        p = reduce - n
        if reduce > len(list(child_data.index)) + len(list(selected_Pop_data.index)):
            print("ERROR")
        else:
            if n > len(child_data):
                n = len(child_data)
                p = reduce - p
            if n == 0:
                n = 1
                p = 1
        d = pd.concat([selected_Pop_data,child_data],axis=0)
        d = d.reset_index(drop=True)
        # print("p:",p)
        # print("n:",n)
        random_number1 = random.sample(list(selected_Pop_data.index),p)
        random_number2 = random.sample(list(range(len(list(selected_Pop_data.index)),len(list(d.index)))),n)
        random_number1.extend(random_number2)
        # print(random_number1)
        for i in random_number1:
          d.drop(i,axis=0,inplace = True)
        d = d.reset_index(drop=True)
        # print("NEW POPULATION")
        # display(d)
        r = d[["Line1","Line2","Line3","Line4"]]
        return d,r
    
    def __init__(self,words_df,n,cross,epoch,z):
        Selection_number = int(round(0.85*n))
        best = []
        assert n>1 ,f"Population {n} cannot be less than or equal to 1"
        # assert n>=Selection_number, f"Cannot select {Selection_number} poems from total population of {n}"
        assert cross>1, f"Minimum number for crossover should be 1"
        # assert 2*n >= Selection_number ,f"Cannot make initial population of {n} if only {Selection_number} parents are selected"

        Population_data = self.create_set_of_poems(n)
        for i in range(epoch):
            Selected_Population_data,selected_polulation_word_array = self.tournament(Population_data,Selection_number,best)
            child_population = self.child_poems(selected_polulation_word_array,cross)
            final_population,final_poems = self.selection_from_child_parents(n,child_population,Population_data)
            if i != epoch-1 or (epoch-1)%10 == 0:
                print("Select best poems")
                display(final_poems)
                Population_data = final_population
                best = (list(map(int,input("\n Enter best poems numbers: ").strip().split())))[:z]
                print("Final poems")
                display(final_poems)

            else:
                print("###################################################")
                print("Last Poems Standing")
                print("###################################################")
                display(final_population)

In [118]:
#4 (words_df,intial_population,crossover,epochs,user_selection_poems_number):
poem = poem_generation(words_df,20,2,2,4)


Select Population


,Line1,Line2,Line3,Line4
0,is is time Dressed,putting well a What,years teetering Larger It,live It As songs
1,From Come above its,air inspired Juggling that,day future mystery Creating,light As the Dram
2,wonder open Threadbare bird,clarinet is was it,are like On In,in death O musician
3,and we environment think,are the We anything,one in poetry where,Not reach we never
4,we not only the,a will flows creating,the Enrol its nature,time In Our Borrowing
5,A bird has to,The now Time memory,A Reflections palpitations We,fades cry gone world
6,high Faith years Coach,every Black yet of,inviolate play all not,so opens white Wings
7,in soldiers living Life,they Life awaits it,Fixed Casino Now life,try aware much all
8,Every and over of,Them the more A,Bang From moved are,cloud the For There
9,From Come above its,air inspired the Dram,we not only the,a will flows creating


Select Population


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,words_array
0,is is time Dressed,putting well a What,years teetering Larger It,live It As songs,5,5,7,4,18,"[is, is, time, Dressed, putting, well, a, What, years, teetering, Larger, It, live, It, As, songs]"
1,From Come above its,air inspired Juggling that,day future mystery Creating,light As the Dram,6,7,9,4,11,"[From, Come, above, its, air, inspired, Juggling, that, day, future, mystery, Creating, light, As, the, Dram]"
2,wonder open Threadbare bird,clarinet is was it,are like On In,in death O musician,8,6,5,6,19,"[wonder, open, Threadbare, bird, clarinet, is, was, it, are, like, On, In, in, death, O, musician]"
3,and we environment think,are the We anything,one in poetry where,Not reach we never,7,7,7,5,9,"[and, we, environment, think, are, the, We, anything, one, in, poetry, where, Not, reach, we, never]"
4,we not only the,a will flows creating,the Enrol its nature,time In Our Borrowing,5,5,6,6,10,"[we, not, only, the, a, will, flows, creating, the, Enrol, its, nature, time, In, Our, Borrowing]"
5,A bird has to,The now Time memory,A Reflections palpitations We,fades cry gone world,4,6,9,5,5,"[A, bird, has, to, The, now, Time, memory, A, Reflections, palpitations, We, fades, cry, gone, world]"
6,high Faith years Coach,every Black yet of,inviolate play all not,so opens white Wings,4,6,7,6,20,"[high, Faith, years, Coach, every, Black, yet, of, inviolate, play, all, not, so, opens, white, Wings]"
7,in soldiers living Life,they Life awaits it,Fixed Casino Now life,try aware much all,6,5,7,6,9,"[in, soldiers, living, Life, they, Life, awaits, it, Fixed, Casino, Now, life, try, aware, much, all]"
8,Every and over of,Them the more A,Bang From moved are,cloud the For There,6,4,6,5,9,"[Every, and, over, of, Them, the, more, A, Bang, From, moved, are, cloud, the, For, There]"
9,From Come above its,air inspired the Dram,we not only the,a will flows creating,6,6,5,5,6,"[From, Come, above, its, air, inspired, the, Dram, we, not, only, the, a, will, flows, creating]"


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,words_array
0,From Come above its,air inspired the Dram,we not only the,a will flows creating,6,6,5,5,6,"[From, Come, above, its, air, inspired, the, Dram, we, not, only, the, a, will, flows, creating]"
1,Every and over of,Them the more A,Bang From moved are,cloud the For There,6,4,6,5,9,"[Every, and, over, of, Them, the, more, A, Bang, From, moved, are, cloud, the, For, There]"
2,and we environment think,are the We anything,one in poetry where,Not reach we never,7,7,7,5,9,"[and, we, environment, think, are, the, We, anything, one, in, poetry, where, Not, reach, we, never]"
3,A bird has to,The now Time memory,A Reflections palpitations We,fades cry gone world,4,6,9,5,5,"[A, bird, has, to, The, now, Time, memory, A, Reflections, palpitations, We, fades, cry, gone, world]"


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,words_array
4,we not only the,a will flows creating,the Enrol its nature,time In Our Borrowing,5,5,6,6,10,"[we, not, only, the, a, will, flows, creating, the, Enrol, its, nature, time, In, Our, Borrowing]"
11,for say take About,shine to Climb your,to As a away,Spirited Focussed Two life,4,5,5,8,10,"[for, say, take, About, shine, to, Climb, your, to, As, a, away, Spirited, Focussed, Two, life]"
1,From Come above its,air inspired Juggling that,day future mystery Creating,light As the Dram,6,7,9,4,11,"[From, Come, above, its, air, inspired, Juggling, that, day, future, mystery, Creating, light, As, the, Dram]"
12,is them everywhere strings,flame was be the,Gaily ship That they,then time A its,8,5,5,4,11,"[is, them, everywhere, strings, flame, was, be, the, Gaily, ship, That, they, then, time, A, its]"
14,years teetering Larger It,live let totality Creatures,Negative fruits we minds,kind It As songs,7,9,7,4,13,"[years, teetering, Larger, It, live, let, totality, Creatures, Negative, fruits, we, minds, kind, It, As, songs]"
10,Inside our Waiting each,told wind where thoughts,from the call one,They its lights Knowledge,6,5,5,6,16,"[Inside, our, Waiting, each, told, wind, where, thoughts, from, the, call, one, They, its, lights, Knowledge]"
15,is unknown its time,fabric Never in Its,is is time Dressed,putting well a What,5,6,5,5,17,"[is, unknown, its, time, fabric, Never, in, Its, is, is, time, Dressed, putting, well, a, What]"
0,is is time Dressed,putting well a What,years teetering Larger It,live It As songs,5,5,7,4,18,"[is, is, time, Dressed, putting, well, a, What, years, teetering, Larger, It, live, It, As, songs]"
2,wonder open Threadbare bird,clarinet is was it,are like On In,in death O musician,8,6,5,6,19,"[wonder, open, Threadbare, bird, clarinet, is, was, it, are, like, On, In, in, death, O, musician]"
13,waters the are gods,into whole Into decay,flies should invent its,from build it our,6,7,5,4,19,"[waters, the, are, gods, into, whole, Into, decay, flies, should, invent, its, from, build, it, our]"



###################################################
Last Poems Standing
###################################################


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,words_array
0,From Come above its,air inspired Juggling that,day future mystery Creating,light As the Dram,6,7,9,4,11,"[From, Come, above, its, air, inspired, Juggling, that, day, future, mystery, Creating, light, As, the, Dram]"
1,high Faith years Coach,every Black yet of,inviolate play all not,so opens white Wings,4,6,7,6,20,"[high, Faith, years, Coach, every, Black, yet, of, inviolate, play, all, not, so, opens, white, Wings]"
2,Every and over of,Them the more A,Bang From moved are,cloud the For There,6,4,6,5,9,"[Every, and, over, of, Them, the, more, A, Bang, From, moved, are, cloud, the, For, There]"
3,Inside our Waiting each,told wind where thoughts,from the call one,They its lights Knowledge,6,5,5,6,16,"[Inside, our, Waiting, each, told, wind, where, thoughts, from, the, call, one, They, its, lights, Knowledge]"
4,is them everywhere strings,flame was be the,Gaily ship That they,then time A its,8,5,5,4,11,"[is, them, everywhere, strings, flame, was, be, the, Gaily, ship, That, they, then, time, A, its]"
5,waters the are gods,into whole Into decay,flies should invent its,from build it our,6,7,5,4,19,"[waters, the, are, gods, into, whole, Into, decay, flies, should, invent, its, from, build, it, our]"
6,On In in death,O musician wonder open,Threadbare to starlight mind,sun of the out,4,8,7,4,8,"[On, In, in, death, O, musician, wonder, open, Threadbare, to, starlight, mind, sun, of, the, out]"
7,me that bite inside,Flicking they are Travellers,adverse About Concentrate high,too human Two There,6,8,9,6,22,"[me, that, bite, inside, Flicking, they, are, Travellers, adverse, About, Concentrate, high, too, human, Two, There]"
8,Time memory A Reflections,palpitations We fades cry,gone world night his,store This known A,8,8,4,5,34,"[Time, memory, A, Reflections, palpitations, We, fades, cry, gone, world, night, his, store, This, known, A]"
9,inspired the Dram we,not only the a,will flows creating Every,and over of Them,6,5,6,5,18,"[inspired, the, Dram, we, not, only, the, a, will, flows, creating, Every, and, over, of, Them]"


In [ ]:
print(list(range(5,8)))

[5, 6, 7]


In [ ]:
a = [2,3,4,5]

for i in a:
    print(i)

2
3
4
5
